# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

In [22]:
nvidia = Website("https://www.nvidia.com")
nvidia.links[:100]

['https://www.nvidia.com',
 '#page-content',
 'https://www.nvidia.com/en-gb/',
 'https://www.nvidia.com/en-gb/geforce/graphics-cards/50-series/',
 'https://www.nvidia.com/en-gb/geforce/laptops/50-series/',
 'https://www.nvidia.com/en-gb/geforce/products/g-sync-monitors/',
 'https://www.nvidia.com/en-gb/studio/laptops-desktops/',
 'https://www.nvidia.com/en-gb/shield/',
 'https://www.nvidia.com/en-gb/ai-on-rtx/',
 'https://www.nvidia.com/en-gb/geforce/laptops/50-series/',
 'https://www.nvidia.com/en-gb/design-visualization/rtx-professional-laptops/',
 'https://www.nvidia.com/en-gb/design-visualization/desktop-graphics/',
 'https://www.nvidia.com/en-gb/project-digits/',
 'https://www.nvidia.com/en-gb/ai-data-science/workstations/',
 'https://www.nvidia.com/en-gb/data-center/',
 'https://www.nvidia.com/en-gb/data-center/grace-cpu/',
 'https://www.nvidia.com/en-gb/data-center/dgx-platform/',
 'https://www.nvidia.com/en-gb/data-center/products/egx/',
 'https://www.nvidia.com/en-gb/edge-comp

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [23]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/Janus-Pro-7B',
 '/deepseek-ai/DeepSeek-V3',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/deepseek-ai/deepseek-vl2-small',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/simplescaling/s1K',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/saiyan-world/Goku-MovieGenBench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gr

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n4 days ago\n•\n3.21M\n•\n8.52k\nZyphra/Zonos-v0.1-hybrid\nUpdated\nabout 12 hours ago\n•\n2.76k\n•\n645\nhexgrad/Kokoro-82M\nUpdated\n11 days ago\n•\n400k\n•\n3.08k\ndeepseek-ai/Janus-Pro-7B\nUpdated\n12 days ago\n•\n391

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of artificial intelligence, fostering a community dedicated to building the future of AI technologies. With a mission to create an open-source platform for machine learning, Hugging Face allows users to host, share, and collaborate on models, datasets, and applications seamlessly. It's a place where innovation meets collaboration, empowering both individuals and organizations to accelerate their machine learning journey.

---

## Our Platform
- **Models**: Explore a diverse collection of over **1 million models**, including state-of-the-art models for various AI applications.
- **Datasets**: Access and share more than **250,000 datasets** dedicated to numerous machine learning tasks including natural language processing (NLP), computer vision, and more.
- **Spaces**: Utilize collaborative Spaces to create and deploy AI applications effortlessly.

With cutting-edge tools such as **Transformers**, **Diffusers**, and various libraries for ease of use, Hugging Face provides the necessary infrastructure for developers and researchers to push the boundaries of AI.

---

## Collaborate and Innovate
**Hugging Face** proudly hosts a vibrant community where:
- **50,000+ Organizations** leverage our platform, including notable names like **Meta, AWS, Google,** and **Microsoft**.
- Users can contribute through posts, updates, and discussions, enhancing the collective knowledge base.
- Individuals can showcase their work, building a compelling portfolio within the AI and ML landscape.

---

## Our Culture
Hugging Face is built on a foundation of collaboration and innovation. We believe that the best advancements in AI come from open discussions and contributions. Our team values:
- **Transparency** and **Inclusivity**: Ensuring everyone has a voice in the community.
- **Continuous Learning**: Encouraging team members to grow through knowledge-sharing and professional development.
- **Community Engagement**: Building relationships with users and developers to co-create AI solutions.

---

## Careers at Hugging Face
At Hugging Face, we are always looking for passionate individuals who are enthusiastic about AI and machine learning. Join our dynamic team and work on exciting projects that influence the future of technology. We offer roles spanning across engineering, research, product design, and community engagement. 

**Benefits of working with us include:**
- An inclusive work environment
- Opportunities for personal and professional development
- A chance to make a significant impact in the AI community

**Interested in joining?** Visit our [Jobs page](https://huggingface.co/jobs) to find current openings.

---

## Connect with Us
For more information, to explore our tools, or to join the community, visit our website [Hugging Face](https://huggingface.co).

Join us in building the future of AI together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

Welcome to **Hugging Face** - where the AI community builds the future.

## About Us
Hugging Face is a groundbreaking collaboration platform, **dedicated to machine learning enthusiasts and professionals**. The platform empowers users to discover, share, and collaborate on **over 1 million models** and **250,000 datasets** for various AI applications. Our mission is to create an open-source ecosystem that facilitates the rapid advancement of AI technology.

## Our Offerings

### Collaborate & Discover
- **Models**: Access a vast collection of machine learning models updated regularly. Collaborate to enhance real-world applications.
- **Datasets**: Browse and share datasets to fuel your AI projects. 
- **Spaces**: Create and deploy applications in our collaborative spaces.

### Enterprise Solutions
We offer dedicated enterprise packages tailored for organizations looking to leverage AI while ensuring **security** and **support**:
- Starting at just **$20/user/month**, gain access to advanced features, dedicated support, and enterprise-grade security.
- Deploy compute solutions easily starting at **$0.60/hour for GPU** usage.

## Our Customers
More than **50,000 organizations** are utilizing Hugging Face, including industry leaders:
- Meta
- Amazon Web Services
- Google
- Microsoft
- Grammarly

## Company Culture
At Hugging Face, we believe in the power of **community-driven innovation**. We foster a collaborative and inclusive environment where creativity thrives. Our team embraces **diversity**, **flexibility**, and **teamwork**, ensuring that everyone contributes to building state-of-the-art machine learning technologies.

## Career Opportunities
Join Hugging Face and be a part of an ambitious team that transforms the AI landscape. We are **always on the lookout** for passionate individuals who wish to make a difference in AI. Explore our job openings on our [careers page](#) and embark on a rewarding journey with us.

## Get Started
Whether you are a developer, researcher, or enterprise, **getting started with Hugging Face** is simple. Sign up [here](#) to create your profile, explore models, and engage with the community.

---

*Connect with us*:
- [Twitter](#)
- [LinkedIn](#)
- [GitHub](#)
- [Discord](#)

Join us in shaping the future of AI, one model at a time!


In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

### Week 1 Challenges

In [26]:
# Using a multi shot system prompt!
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in the following examples:"
link_system_prompt += """\nFor the first example, given the following list of website links:

[
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/Janus-Pro-7B',
 '/deepseek-ai/DeepSeek-V3',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/deepseek-ai/deepseek-vl2-small',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/join',
 '/pricing',
 '/enterprise',
 '/datasets',
]


You should respond with a json object like the following:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "enterprise page": "url": "https://another.full.url/enterprise"}
    ]
}
"""
link_system_prompt += """\n For the second example, given a list of website links like the following:

[
 'https://www.nvidia.com',
 '#page-content',
 'https://www.nvidia.com/en-gb/',
 'https://www.nvidia.com/en-gb/geforce/graphics-cards/50-series/',
 'https://www.nvidia.com/en-gb/geforce/laptops/50-series/',
 'https://www.nvidia.com/en-gb/geforce/products/g-sync-monitors/',
 'https://www.nvidia.com/en-gb/studio/laptops-desktops/',
 'https://www.nvidia.com/en-gb/shield/',
 'https://www.nvidia.com/en-gb/ai-on-rtx/',
 'https://www.nvidia.com/en-gb/geforce/laptops/50-series/',
 'https://www.nvidia.com/en-gb/design-visualization/rtx-professional-laptops/',
 'https://www.nvidia.com/en-gb/design-visualization/desktop-graphics/',
 'https://www.nvidia.com/en-gb/project-digits/',
 'https://www.nvidia.com/en-gb/ai-data-science/workstations/',
 'https://www.nvidia.com/en-gb/data-center/',
 'https://www.nvidia.com/en-gb/data-center/grace-cpu/',
 'https://www.nvidia.com/en-gb/data-center/dgx-platform/',
 'https://www.nvidia.com/en-gb/data-center/products/egx/',
 'https://www.nvidia.com/en-gb/edge-computing/products/igx/',
 'https://www.nvidia.com/en-gb/data-center/hgx/',
 'https://www.nvidia.com/en-gb/data-center/products/mgx/',
 'https://www.nvidia.com/en-gb/omniverse/platform/ovx/',
 'https://www.nvidia.com/en-gb/self-driving-cars/simulation/',
 'https://www.nvidia.com/en-gb/networking/',
 'https://www.nvidia.com/en-gb/networking/products/data-processing-unit/',
 'https://www.nvidia.com/en-gb/networking/products/ethernet/',
 'https://www.nvidia.com/en-gb/networking/products/infiniband/',
 'https://www.nvidia.com/en-gb/geforce/graphics-cards/50-series/',
 'https://www.nvidia.com/en-gb/design-visualization/rtx/',
 'https://www.nvidia.com/en-gb/data-center/technologies/hopper-architecture/',
]


You should respond with a json object like the following:

"links": [
        {"type": "shield page", "url": "https://www.nvidia.com/en-gb/shield/"},
        {"type": "data center page": "url": "https://www.nvidia.com/en-gb/data-center/"},
        {"type": "networking page": "url": "https://www.nvidia.com/en-gb/networking"},
        {"type": "ethernet prodcut page": "url": "https://www.nvidia.com/en-gb/networking/products/ethernet"}
    ]

"""

In [27]:
# Using the multi-shot prompt defined above!
get_links("https://www.nvidia.com/en-gb/")

{'links': [{'type': 'about page',
   'url': 'https://www.nvidia.com/en-gb/about-nvidia/'},
  {'type': 'careers page',
   'url': 'https://www.nvidia.com/en-gb/about-nvidia/careers/'},
  {'type': 'executive insights page',
   'url': 'https://www.nvidia.com/en-gb/executive-insights/'},
  {'type': 'investor relations page',
   'url': 'https://investor.nvidia.com/home/default.aspx'},
  {'type': 'company policies page',
   'url': 'https://www.nvidia.com/en-gb/about-nvidia/company-policies/'},
  {'type': 'community page', 'url': 'https://www.nvidia.com/en-us/community/'},
  {'type': 'partnerships page',
   'url': 'https://www.nvidia.com/en-gb/about-nvidia/partners/'}]}

In [28]:
stream_brochure("Nvidia", "https://nvidia.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/careers/'}, {'type': 'executive insights page', 'url': 'https://www.nvidia.com/en-gb/executive-insights/'}, {'type': 'foundation page', 'url': 'https://www.nvidia.com/en-us/foundation/'}, {'type': 'research page', 'url': 'https://www.nvidia.com/en-us/research/'}, {'type': 'csr page', 'url': 'https://www.nvidia.com/en-us/csr/'}, {'type': 'events page', 'url': 'https://www.nvidia.com/en-gb/events/'}, {'type': 'investor relations page', 'url': 'https://investor.nvidia.com/home/default.aspx'}, {'type': 'training page', 'url': 'https://www.nvidia.com/en-gb/training/'}]}


# NVIDIA Company Brochure

## Overview
NVIDIA is a world leader in artificial intelligence computing, known for its innovative products and technologies designed to accelerate computing across various domains. From gaming to healthcare, NVIDIA's cutting-edge solutions drive advancements in graphics processing, cloud computing, and AI.

## Products & Solutions
NVIDIA's extensive product portfolio includes:

- **Graphics Cards**: GeForce for gaming and professional use, RTX technology for real-time ray tracing.
- **Cloud & Data Center Solutions**: DGX and EGX platforms for AI workloads and accelerated computing.
- **AI Solutions**: Tools for AI inference, machine learning, generative AI, and more.
- **Automotive Technologies**: Self-driving vehicle solutions, in-vehicle computing, and automated driving systems.

## Customer Focus
NVIDIA serves a diverse range of industries, including:
- **Gaming**: Powering immersive gaming experiences.
- **Healthcare**: Enhancing research and diagnostics with AI.
- **Automotive**: Developing self-driving technology and smart vehicle systems.
- **Supercomputing**: Supporting complex simulation and modeling workloads.
- **Media & Entertainment**: Revolutionizing content creation and visualization.

## Company Culture
At NVIDIA, innovation is at the core of the company culture. Employees enjoy a collaborative environment that encourages creativity and critical thinking. NVIDIA values diversity and inclusivity, believing that a wide range of perspectives leads to better ideas and solutions. Employees are motivated to push the boundaries of technology through continuous learning and development.

## Careers & Opportunities
NVIDIA is always on the lookout for passionate individuals who are eager to make a difference. Here are some career opportunities:

- **Engineering**: Develop the next generation of GPUs and AI technologies.
- **Data Science**: Work on cutting-edge analytics and AI modeling.
- **Product Management**: Lead innovative product strategy and execution.
- **Sales and Marketing**: Drive engagement and outreach for NVIDIA’s solutions.

NVIDIA provides competitive salaries, comprehensive benefits, and ample opportunities for professional growth. Join us to be a part of the AI revolution!

## Conclusion
Whether you are a prospective customer seeking advanced technology solutions, an investor looking for a robust business with high growth potential, or a recruit aspiring to innovate in the tech industry, NVIDIA is dedicated to shaping the future of computing and AI.

Discover more at [NVIDIA](https://www.nvidia.com).

--- 

Feel free to customize this brochure further as per your needs!

In [46]:
# Changing the system and user prompt to request the Brochure in a particular format!
formatted_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Return the Brocher in the following format using only three sections to effectively convey the most relevant information:\n"
formatted_system_prompt += "# <Brochure Title>\n***Created by BrochureBot***\n\n<Section 1 Title>\n<text>\n\n<Section 2 Title>\n<text>\n\n<Section 3 Title>\n<text>"

def get_formatted_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += f"Make sure to follow the correct format of the Brochure."
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def create_formatted_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": formatted_system_prompt},
            {"role": "user", "content": get_formatted_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

def stream_formatted_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": formatted_system_prompt},
            {"role": "user", "content": get_formatted_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [47]:
stream_formatted_brochure('Nvidia', 'https://www.nvidia.com')

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/careers/'}, {'type': 'executive insights page', 'url': 'https://www.nvidia.com/en-gb/executive-insights/'}, {'type': 'programs page', 'url': 'https://www.nvidia.com/en-us/programs/isv/'}, {'type': 'training page', 'url': 'https://www.nvidia.com/en-gb/training/'}]}


# Discover NVIDIA: Pioneers in AI Computing
***Created by BrochureBot***

## Innovation and Leadership
NVIDIA stands at the forefront of artificial intelligence and computing technology. Renowned as the world leader in AI computing, NVIDIA develops powerful GPUs, high-performance computing solutions, and advanced AI platforms. Their product range caters to various industries including automotive, healthcare, gaming, and data-centric sectors. From desktop graphics cards to AI inference and cloud solutions, NVIDIA empowers businesses with robust technology to drive innovation.

## Company Culture and Values
At NVIDIA, a culture of continuous learning, diversity, and collaboration thrives. The company encourages creativity and innovation among its employees, fostering an environment where ideas can flourish. Employees are passionate about technology and dedicated to transforming the way we interact with the world through AI and graphics. NVIDIA believes in inclusivity and supports initiatives that promote well-being, making it a great place to work for enthusiasts and experts alike.

## Careers at NVIDIA
NVIDIA offers a variety of career opportunities for individuals seeking to make an impact in the tech world. Whether you're a developer, researcher, designer, or in business operations, NVIDIA seeks talented and innovative individuals ready to push the boundaries of technology. Employees benefit from competitive compensation, professional development, and the chance to work on groundbreaking projects that shape the future of artificial intelligence and computer graphics. Join the NVIDIA team to be part of a legacy that defines technological progress.

In [48]:
stream_formatted_brochure('Anduril', 'https://www.anduril.com/')

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anduril.com/'}, {'type': 'careers page', 'url': 'https://www.anduril.com/careers/'}, {'type': 'internships page', 'url': 'https://www.anduril.com/internships-at-anduril/'}, {'type': 'mission page', 'url': 'https://www.anduril.com/mission/'}, {'type': 'newsroom page', 'url': 'https://www.anduril.com/newsroom/'}, {'type': 'media page', 'url': 'https://www.anduril.com/media/'}]}


# Anduril Industries: Pioneering Defense Technology
***Created by BrochureBot***

## Innovative Military Solutions
Anduril Industries is transforming U.S. and allied military capabilities through advanced technology. Our flagship system, Lattice, powers a family of autonomous systems that provide robust command and control, mission autonomy, and persistent security across air, land, and sea. This includes a range of products such as unmanned aerial vehicles (UAVs) like the Bolt and Barracuda, and advanced underwater vehicles like Dive-XL, engineered for various intelligence, surveillance, and reconnaissance missions. Additionally, we supply next-generation solid rocket motors, ensuring our military remains at the forefront of innovation.

## Empowering Company Culture
At Anduril, we embrace a culture that values collaboration between skilled professionals and veterans who have firsthand experience with military challenges. We seek thinkers and doers who can work interdependently, encouraging a dynamic work environment that promotes creative problem solving. With a commitment to hiring emerging talent and integrating efforts through initiatives like the SkillBridge program, we equip our team members with the tools needed to drive technological advancements in defense.

## Career Opportunities and Growth
Anduril is hiring! We offer a variety of career paths for talented individuals passionate about technology and defense. Our positions include roles in engineering, operations, and product development, providing opportunities for impactful work that supports warfighters. Check our careers page for open roles and discover how you can join our mission to enhance military capabilities with cutting-edge technology. Join us at Anduril, where your work matters and drives change.

In [49]:
# Make a third call to an LLM to tranlate the entire brochure to French! 
translator_system_prompt = "You are an assistant which specialises in translating business brochures into French.\
You will be supplied with a Brochure in markdown format, translate the entirety of the Brochure into French.\
Return the translated brochure in markdown format and respecting the original formatting of the Brochure."

def get_translator_user_prompt(company_name, url):
    user_prompt = f"The following is a bussiness brochure for the company {company_name}."
    user_prompt += f"Please translate the entirety of the brochure into French, make sure to respect the original markdown formatting."
    user_prompt += f"Original Brochure:\n\n"
    user_prompt += create_formatted_brochure(company_name, url)
    return user_prompt[:10000] # Truncate if more than 10000 chars just in case


def stream_translated_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translator_system_prompt},
            {"role": "user", "content": get_translator_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response = ''
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [50]:
stream_translated_brochure('Nvidia', 'https://www.nvidia.com')

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-gb/about-nvidia/careers/'}, {'type': 'executive insights page', 'url': 'https://www.nvidia.com/en-gb/executive-insights/'}, {'type': 'events page', 'url': 'https://www.nvidia.com/en-us/events/'}, {'type': 'research page', 'url': 'https://www.nvidia.com/en-us/research/'}, {'type': 'corporate brief pdf', 'url': 'https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/corporate-nvidia-in-brief.pdf'}, {'type': 'NVIDIA story pdf', 'url': 'https://images.nvidia.com/pdf/NVIDIA-Story.pdf'}, {'type': 'training page', 'url': 'https://www.nvidia.com/en-gb/training/'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Découvrez NVIDIA : À la tête de l'avenir de l'informatique AI
***Créé par BrochureBot***

## Aperçu de l'entreprise
NVIDIA se positionne comme un leader mondial dans l'informatique par intelligence artificielle, pionnier des avancées dans de nombreuses industries, notamment le jeu vidéo, les centres de données, l'automobile, la santé, et bien d'autres. Avec une gamme diversifiée de produits tels que des cartes graphiques GeForce, des plateformes AI puissantes et des services cloud dédiés, NVIDIA fournit des solutions à la pointe de la technologie, conçues pour répondre aux exigences de l'informatique moderne. Nos principaux domaines d'intervention incluent l'IA et l'apprentissage automatique, l'informatique haute performance, la robotique et le métavers en évolution.

## Culture d'entreprise
Chez NVIDIA, nous cultivons une culture d'entreprise innovante et inclusive, animée par une passion commune pour la technologie et l'excellence. Nous valorisons la créativité, encourageons la collaboration et privilégions la croissance professionnelle. Notre équipe est composée de talents divers venus du monde entier, travaillant ensemble pour résoudre certains des problèmes les plus difficiles de la planète. Nous croyons en la nécessité de maintenir un environnement de travail qui inspire et habilite nos employés, soutenu par des programmes qui promeuvent la diversité, la durabilité et l'équilibre entre vie professionnelle et vie privée.

## Carrières chez NVIDIA
Rejoignez-nous chez NVIDIA et faites partie d'une équipe pionnière qui façonne l'avenir de la technologie. Nous offrons des opportunités de carrière passionnantes dans divers domaines, allant de l'ingénierie aux ventes et au marketing. Avec de nombreuses initiatives pour développer les compétences et valoriser les talents, nous nous engageons à recruter les meilleures esprits et à favoriser leur croissance au sein de l'entreprise. Explorez votre potentiel avec nous et contribuez à créer des technologies révolutionnaires qui impactent des millions de personnes dans le monde. 

---

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>